# Import

I think many people are having problems with the target variable this is where i will explain it nicely with code

In [ ]:
import numpy as np
import pandas as pd

# Load Data 

In [ ]:
df=pd.read_csv('../input/g-research-crypto-forecasting/train.csv')
df.head()

crop the asset id

In [ ]:
df=df[df['Asset_ID']==0]
df['date']=pd.to_datetime(df['timestamp'],unit='s')

In [ ]:
df.head()

In [ ]:
dfnumpy=np.array(df[['Close','Target']])
dfnumpy[[0,1]]

In [ ]:
asset=pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')
asset

# Lets peel the onion now

First computation
$$R^a(t) = log (P^a(t+16)\ /\ P^a(t+1))$$

First lets understand $P^a$ it means it means that P is the Close price or the price at the end of the minute for the asset id $a$ t is our current minute for which we want to compute the targets now $P^a$ are prices for asset id $a$ so t+16 is the price after 16 minutes from the current timestamp while t+1 is just the next timestamp , so $P^a(t+16)$ is the timestamp after the end of 15 minutes and $P^a(t+1)$ is at the start of the minute taking these two future price values we calculates $R^a$ with given formula

Code is given below

In [ ]:
t=0
num=dfnumpy[t+16,0]
denom=dfnumpy[t+1,0]
r=np.log(num/denom)
r

Second compuatition [can be ignored also],

$$M(t) = \frac{\sum_a w^a R^a(t)}{\sum_a w^a}$$

in this formula we basically take a weighted average of the target values but as we can cancel out $w^a$ we can skip this we should have used this if we had many samples with different weights but here this can be ignored  as here $\sum_a w^a$ is just $w^a$ with no change for only one sample so M=R

In [ ]:
M=r

# Beta calculation 

$\beta$ is just the rolling average of $M^2$ with 3750 minutes so lets say i compute R value not just for one sample but for many samples if we define that by $M$ then the code is simple in pandas
```df['beta']=(df['M']**2.rolling.mean(3750).mean()```  https://pandas.pydata.org/docs/reference/api/pandas.core.window.rolling.Rolling.mean.html 

but as where we just have R for one sample or one minute we cannot get 3750 minutes so we were not able to compute this here and this will result into close but not exact measurements

In [ ]:
beta=M**2

# Target compuation come on this is simple
$$\text{Target}^a(t) = R^a(t) - \beta^a M(t)$$

just plug in the values and compute it

In [ ]:
Target=r-beta*M
Target